In [1]:
import backtrader as bt
import alpaca_backtrader_api as api
import alpaca_trade_api as tradeapi
import sys
sys.path.insert(0, '../')
import pandas as pd
import talib as ta
import numpy as np
from config import key_id, secret_key

In [2]:
#import cross signal generator
from cross_signal import *

In [3]:
import random
from datetime import datetime, timedelta


# or a function
def gen_datetime(min_year=1900, max_year=2019):
    # generate a datetime in format yyyy-mm-dd hh:mm:ss.000000
    start = datetime(min_year, 1, 1, 00, 00, 00)
    years = max_year - min_year + 1
    end = start + timedelta(days=365 * years)
    return start + (end - start) * random.random()

In [4]:
#trading data is called in the form of JSON
#https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/2019-01-01/2019-02-01?apiKey=


api = tradeapi.REST(key_id, secret_key)

In [5]:
#generate list of random days into list
day_list=[]
day_after_list=[]
for i in range(500):
    rand_day=gen_datetime(2009)
    day_list.append(rand_day.strftime("%Y-%m-%d"))
    day_after=rand_day + timedelta(days=14)
    day_after_list.append(day_after.strftime("%Y-%m-%d"))

In [6]:
day_list[0]

'2010-07-01'

In [7]:
#generate random list of symbols
sym_list=['JACK']
# ,'AAPL','TSLA','JACK'
#for i in range(10):
    #rand_sym=random.random(range=len(symbols.csv))
    #symlist.append(rand_sym)

In [8]:
#Generate random day data for
df_list=[]
for sym in sym_list: 
    data_day_list=[]
    for i in range(len(day_list)):
        data_sym = api.polygon.historic_agg_v2(sym, 1, 'minute', _from=day_list[i], to=day_after_list[i]).df
        data_day_list.append(data_sym)
    df_list.append(data_day_list)

In [9]:
cross_sign=crossed_below(df_list[0][0].high,60)
cross_sign[0]

False

# Testing

In [104]:
#feed strat to cerebro

class volumestrat(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=4)
        self.emaclose=bt.indicators.EMA(self.data.close, period=4)
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        
        self.mfi=bt.talib.MFI(self.data.high,self.data.low,self.data.close,self.data.volume, timeperiod=14 )
        self.ultosc=bt.talib.ULTOSC(self.data.high,self.data.low,self.data.close)
        
        self.minusdi=bt.talib.MINUS_DI(self.data.high,self.data.low,self.data.close)
        self.plusdi=bt.talib.PLUS_DI(self.data.high,self.data.low,self.data.close)
     
        
        #creat stochastic momentum indicators 
        #1 for down, -1 for up
        #self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.mfi < 85 and \
            self.ultosc < 85 and \
            self.minusdi > 25: \
            #self.minusdi > self.plusdi:
                self.buy(size=20)
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.mfi > 85  and \
            self.ultosc > 85  and \
            self.plusdi > 25:
                self.sell(size=20)


In [131]:
#feed strat to cerebro

class volumestrat4(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=4)
        self.emaclose=bt.indicators.EMA(self.data.close, period=4)
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        
        self.mfi=bt.talib.MFI(self.data.high,self.data.low,self.data.close,self.data.volume, timeperiod=14 )
        self.ultosc=bt.talib.ULTOSC(self.data.high,self.data.low,self.data.close)
        
        self.minusdi=bt.talib.MINUS_DI(self.data.high,self.data.low,self.data.close)
        self.plusdi=bt.talib.PLUS_DI(self.data.high,self.data.low,self.data.close)
     
        
        #creat stochastic momentum indicators 
        #1 for down, -1 for up
        #self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.mfi < 30 and \
            self.ultosc < 30 and \
            self.minusdi > 20: \
            #self.minusdi > self.plusdi:
                self.buy(size=20)
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.mfi > 85  and \
            self.ultosc > 85  and \
            self.plusdi > 20:
                self.sell(size=20)


In [148]:
#feed strat to cerebro

class volumestrat2(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=4)
        self.emaclose=bt.indicators.EMA(self.data.close, period=4)
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        
        self.mfi=bt.talib.MFI(self.data.high,self.data.low,self.data.close,self.data.volume, timeperiod=14 )
        self.ultosc=bt.talib.ULTOSC(self.data.high,self.data.low,self.data.close)
        
        self.minusdi=bt.talib.MINUS_DI(self.data.high,self.data.low,self.data.close)
        self.plusdi=bt.talib.PLUS_DI(self.data.high,self.data.low,self.data.close)
     
        
        #creat stochastic momentum indicators 
        #1 for down, -1 for up
        #self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.mfi < 20 and \
            self.ultosc < 20 and \
            self.minusdi > 25 and \
            self.minusdi > self.plusdi:
                self.buy(size=20)
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.mfi > 80  and \
            self.ultosc > 80  and \
            self.minusdi > 25 and \
            self.minusdi < self.plusdi:
                self.sell(size=20)


In [1]:
#feed strat to cerebro

class volumestrat7(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=4)
        self.emaclose=bt.indicators.EMA(self.data.close, period=4)
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        
        self.mfi=bt.talib.MFI(self.data.high,self.data.low,self.data.close,self.data.volume, timeperiod=14 )
        self.ultosc=bt.talib.ULTOSC(self.data.high,self.data.low,self.data.close)
        
        self.minusdi=bt.talib.MINUS_DI(self.data.high,self.data.low,self.data.close)
        self.plusdi=bt.talib.PLUS_DI(self.data.high,self.data.low,self.data.close)
     
        
        #creat stochastic momentum indicators 
        #1 for down, -1 for up
        #self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.mfi < 20 and \
            self.ultosc < 20 and \
            self.minusdi > 25 and \
            self.minusdi > self.plusdi:
                self.buy(size=20)
                
                
        else:
            if self.mfi > 80  and \
            self.ultosc > 80  and \
            self.minusdi > 25 and \
            self.minusdi < self.plusdi: 
                self.sell(size=20)


NameError: name 'bt' is not defined

In [205]:
#feed strat to cerebro

class volumestrat4(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=4)
        self.emaclose=bt.indicators.EMA(self.data.close, period=4)
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        
        self.mfi=bt.talib.MFI(self.data.high,self.data.low,self.data.close,self.data.volume, timeperiod=14 )
        self.ultosc=bt.talib.ULTOSC(self.data.high,self.data.low,self.data.close)
        
        self.minusdi=bt.talib.MINUS_DI(self.data.high,self.data.low,self.data.close)
        self.plusdi=bt.talib.PLUS_DI(self.data.high,self.data.low,self.data.close)
     
        
        #creat stochastic momentum indicators 
        #1 for down, -1 for up
        #self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.mfi < 25 and \
            self.ultosc < 25: \
            #self.minusdi > self.plusdi:
                self.buy(size=20)
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.mfi > 87  and \
            self.ultosc > 87:
                self.sell(size=20)


In [221]:
#feed strat to cerebro

class volumestrat2(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=4)
        self.emaclose=bt.indicators.EMA(self.data.close, period=4)
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        
        self.mfi=bt.talib.MFI(self.data.high,self.data.low,self.data.close,self.data.volume, timeperiod=14 )
        self.ultosc=bt.talib.ULTOSC(self.data.high,self.data.low,self.data.close)
        
        self.minusdi=bt.talib.MINUS_DI(self.data.high,self.data.low,self.data.close)
        self.plusdi=bt.talib.PLUS_DI(self.data.high,self.data.low,self.data.close)
     
        
        #creat stochastic momentum indicators 
        #1 for down, -1 for up
        #self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.mfi < 15 and \
            self.ultosc < 25 and \
            self.minusdi > 25:
                self.buy(size=20)
                
                
        else:
            if self.mfi > 90  and \
            self.ultosc > 80  and \
            self.minusdi > 25:
                self.sell(size=20)


In [258]:
#feed strat to cerebro

class oscillator_mfi(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=4)
        self.emaclose=bt.indicators.EMA(self.data.close, period=4)
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        
        
        self.mfi=bt.talib.MFI(self.data.high,self.data.low,self.data.close,self.data.volume, timeperiod=14 )
        
        #crossing signal
        self.adosc=bt.talib.ADOSC(self.data.high,self.data.low,self.data.close,self.data.volume, fastperiod=3, slowperiod=10) 
        #1 for down, -1 for up
        self.cross_sig=bt.indicators.CrossOver(self.adosc, 0)
        
        
        self.emalow=bt.indicators.EMA(self.data.low, period=4)
        
        self.minusdi=bt.talib.MINUS_DI(self.data.high,self.data.low,self.data.close)
        self.plusdi=bt.talib.PLUS_DI(self.data.high,self.data.low,self.data.close)
     
        

        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.cross_sig== -1 and \
            self.mfi < 30: \
            #self.minusdi > self.plusdi:
                self.buy(size=20)
                
                
        else:
            if self.cross_sig== 1 and \
            self.mfi > 85:
                self.sell(size=20)


In [254]:
"""self.cross_sig_below=crossed_below(self.adosc,0)
self.cross_sig_above=crossed_above(self.adosc,0)
        
        
    def next(self):
        if not self.position: #not in the market
            if self.cross_sig_above== True and \
            self.mfi < 30: \
            #self.minusdi > self.plusdi:
                self.buy(size=20)
                
                
        else:
            if self.cross_sig_below== True and \
            self.mfi > 85:
                self.sell(size=20)"""

'self.cross_sig_below=crossed_below(self.adosc,0)\nself.cross_sig_above=crossed_above(self.adosc,0)\n        \n        \n    def next(self):\n        if not self.position: #not in the market\n            if self.cross_sig_above== True and             self.mfi < 30:             #self.minusdi > self.plusdi:\n                self.buy(size=20)\n                \n                \n        else:\n            if self.cross_sig_below== True and             self.mfi > 85:\n                self.sell(size=20)'

In [255]:
cross_sign=crossed_below(df_list[0][0].high,60)
cross_sign[0]

False

In [256]:
#feed strat to cerebro

class Scalp4(bt.Strategy):
    """
        this strategy is based around the idea of generating a lot of potential buys and make tiny profits on each trade
        we recommend to have at least 60 parallel trades at any time to cover non avoidable losses.
        Recommended is to only sell based on ROI for this strategy
    """ 
#populate a dataframe with indicators
    
    
    
#------------------------------------strategy starts here--------------------------------------------------------------    
    def __init__(self):
        #populate exponential moving averages
        self.emahigh=bt.indicators.EMA(self.data.high, period=5)
        self.emaclose=bt.indicators.EMA(self.data.close, period=5)
        self.emalow=bt.indicators.EMA(self.data.low, period=5)
        self.rsi=bt.talib.RSI(self.data.close, timeperiod=14 )
        self.cci=bt.talib.CCI(self.data.high,self.data.low,self.data.close, timeperiod=14 )
        
        #macd indicators
        #self.macd_arr=bt.indicators.macd(self.data.close)
        
        #self.macd=self.macd_arr[0]
        #self.macdsignal=self.macd_arr[1]
        
        
        
        #creat stochastic momentum indicators 
        self.stoch=bt.talib.STOCHF(self.data.high, self.data.low, self.data.close, \
                                   fastk_period=5, slowk_period=3, slowd_period=3)
        self.fastk=self.stoch.fastk
        self.fastd=self.stoch.fastd
        #1 for down, -1 for up
        self.fastcross=bt.indicators.CrossOver(self.fastk, self.fastd)
        
        
        #create Average Directional Movement Index
        self.adx=bt.talib.ADX(self.data.high, self.data.low, self.data.close, timeperiod=14)
        
        
# if open score is less than ema_low, adx is greater than 30, fastk fastd less than 30, then buy
#uses indicators to determine buy and sell

    def next(self):
        if not self.position: #not in the market
            if self.data.open < self.emalow and \
            self.adx > 30 and \
            self.rsi < 29  and \
            self.fastd < 30 and \
            self.fastk > self.fastd and \
            self.fastk < 30:
                self.buy(size=30)
                
                
        else:
            if self.data.open >= self.emahigh and \
            self.fastd > 70  and \
            self.rsi > 80  and \
            self.fastk > 70:
                self.sell(size=30)


In [2]:
# Run over everything
SQN_dict_list=[] 
tradeanalysis_dict_list=[]
ret_dict_list=[]
trans_dict_list=[]
pnl_list=[]

for i in range(len(df_list[0])):
    #create backtrader
    cerebro = bt.Cerebro()

    #cerebro.addstrategy(SmaCross)
    #cerebro.addstrategy(Scalp)

    #Add slippage
    cerebro.broker = bt.brokers.BackBroker(slip_perc=0.005) 

    #set up cerebro

    cerebro.broker.setcash(100000)

    cerebro.broker.setcommission(commission=0.0)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=50)

    # Add the analyzers we are interested in
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
    cerebro.addanalyzer(bt.analyzers.Returns, _name="ret")
    cerebro.addanalyzer(bt.analyzers.PeriodStats, _name="pstat")
    cerebro.addanalyzer(bt.analyzers.Transactions, _name="trans")


    # Add the observers we are interested in 
    #this adds more things to the plot on top of the stocks chart
    cerebro.addobserver(bt.observers.Broker)
    cerebro.addobserver(bt.observers.BuySell)
    
    
    
    
    cerebro.addstrategy(oscillator_mfi)
    datacere = bt.feeds.PandasData(dataname=df_list[0][i])
    cerebro.adddata(datacere,name=day_list[i])
#    print(day_list[i]+' added')
    test=cerebro.run()
    
    
    
    
    
    SQN_dict=test[0].analyzers.sqn.get_analysis() 
    SQN_dict_list.append(SQN_dict)
    
    tradeanalysis_dict=test[0].analyzers.ta.get_analysis()
    tradeanalysis_dict_list.append(tradeanalysis_dict)
    
    ret_dict=test[0].analyzers.ret.get_analysis()
    ret_dict_list.append(ret_dict)
    
    trans_dict=test[0].analyzers.trans.get_analysis()
    trans_dict_list.append(trans_dict_list)
    

NameError: name 'df_list' is not defined

In [3]:
SQN_total=0
for i in range(len(SQN_dict_list)):    
    SQN_total=SQN_total+SQN_dict_list[i]['sqn']

SQN_avg=SQN_total/len(SQN_dict_list)

ZeroDivisionError: division by zero

In [4]:
SQN_avg

NameError: name 'SQN_avg' is not defined

In [5]:
tradeanalysis_dict_list[0]

IndexError: list index out of range

In [6]:
win_total=0
total_won_pnl=0
avg_won=0

lost_total=0
total_lost_pnl=0
avg_lost=0

total_pnl=0
total_avg=0

for i in range(len(df_list[0])):
    try:
        win_total=win_total+tradeanalysis_dict_list[i]['won']['total']
        total_won_pnl=total_won_pnl+tradeanalysis_dict_list[i]['won']['pnl']['total']
        avg_won=avg_won+tradeanalysis_dict_list[i]['won']['pnl']['average']

        lost_total=lost_total+tradeanalysis_dict_list[i]['lost']['total']
        total_lost_pnl=total_lost_pnl+tradeanalysis_dict_list[i]['lost']['pnl']['total']
        avg_lost=avg_lost+tradeanalysis_dict_list[i]['lost']['pnl']['average']

        total_pnl=total_pnl+tradeanalysis_dict_list[i]['pnl']['gross']['total']
        total_avg=total_avg+tradeanalysis_dict_list[i]['pnl']['gross']['average']
    except:
        pass


NameError: name 'df_list' is not defined

In [7]:
win_total , total_won_pnl , avg_won 

(0, 0, 0)

In [265]:
lost_total , total_lost_pnl , avg_lost

(988, -26461.072000000004, -12481.48570000001)

In [266]:
total_pnl , total_avg

(7083.223999999994, 863.2129500000007)

In [267]:
win_total/(win_total+lost_total)

0.6055888223552894

In [268]:
total_pnl/(total_pnl+lost_total)

0.8775898178516666

In [269]:
avg_won/(-avg_lost)

0.8635478203235784

In [270]:
# print the SQN analyzers  SystemQualityNumber
#test[0] is the first strategy used. For multiple strategies, use corresponding number in the list.
#SQN_dict=test[0].analyzers.sqn.get_analysis() 
#tradeanalysis_dict=test[0].analyzers.ta.get_analysis()
#ret_dict=test[0].analyzers.ret.get_analysis()
#trans_dict=test[0].analyzers.trans.get_analysis()


In [271]:
trans_dict

OrderedDict([(datetime.datetime(2011, 11, 21, 21, 46),
              [[20, 52.6329, 0, '2011-11-19', -1052.658]]),
             (datetime.datetime(2011, 11, 21, 22, 12),
              [[-20, 52.7157, 0, '2011-11-19', 1054.3139999999999]]),
             (datetime.datetime(2011, 11, 21, 23, 11),
              [[20, 52.7157, 0, '2011-11-19', -1054.3139999999999]]),
             (datetime.datetime(2011, 11, 22, 14, 31),
              [[-20, 53.1186, 0, '2011-11-19', 1062.372]]),
             (datetime.datetime(2011, 11, 22, 16, 5),
              [[20, 53.1146, 0, '2011-11-19', -1062.2920000000001]]),
             (datetime.datetime(2011, 11, 29, 19, 37),
              [[-20, 53.4643, 0, '2011-11-19', 1069.286]]),
             (datetime.datetime(2011, 11, 29, 21, 51),
              [[20, 53.2143, 0, '2011-11-19', -1064.286]])])

In [272]:
len(trans_dict)

7